<a href="https://colab.research.google.com/github/zotikova/usefull_files/blob/main/video_upscaler_with_noise_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title  ## Step 1: Mount Google Drive and Install Dependencies
import pathlib
from google.colab import drive

drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
!apt install ffmpeg                   &> /dev/null
!apt install libmagic1 python3-yaml   &> /dev/null
!apt install libvulkan-dev            &> /dev/null

!wget https://github.com/k4yt3x/video2x/archive/refs/tags/5.0.0-beta5.tar.gz &> /dev/null 
!tar -xvf 5.0.0-beta5.tar.gz &> /dev/null
!rm -rf 5.0.0-beta5.tar.gz

%cd video2x-5.0.0-beta5/video2x
!pip install -r /content/video2x-4.8.1/src/requirements.txt &> /dev/null

if pathlib.Path.is_dir(pathlib.Path('/content/conf_files')) is False:
  !mkdir /content/conf_files

%cd /content/conf_files
!wget https://raw.githubusercontent.com/zotikova/usefull_files/main/video2x.yaml

%cd /content/
!wget https://github.com/nihui/realsr-ncnn-vulkan/releases/download/20200818/realsr-ncnn-vulkan-20200818-linux.zip &> /dev/null
!unzip realsr-ncnn-vulkan-20220728-linux.zip &> /dev/null
!wget https://github.com/nihui/waifu2x-ncnn-vulkan/releases/download/20200818/waifu2x-ncnn-vulkan-20200818-linux.zip &> /dev/null
!unzip waifu2x-ncnn-vulkan-20220728-linux.zip &> /dev/null
!wget https://github.com/nihui/realcugan-ncnn-vulkan/releases/download/20220728/realcugan-ncnn-vulkan-20220728-ubuntu.zip &> /dev/null
!unzip realcugan-ncnn-vulkan-20220728-ubuntu.zip &> /dev/null

!rm *-linux.zip
!pip install -U PyYAML &> /dev/null

/content/video2x-4.8.1/src
/content/conf_files
--2022-10-13 08:23:40--  https://raw.githubusercontent.com/zotikova/usefull_files/main/video2x.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5317 (5.2K) [text/plain]
Saving to: ‘video2x.yaml’

video2x.yaml        100%[===================>]   5.19K  --.-KB/s    in 0s      

2022-10-13 08:23:40 (69.3 MB/s) - ‘video2x.yaml’ saved [5317/5317]

/content


In [ ]:
#@title ## Step 2: Import Input Video

if pathlib.Path.is_dir(pathlib.Path('/content/videos')) is False:
  !mkdir /content/videos
%cd /content/videos

from google.colab import files

uploaded = files.upload()
%cd ..

/content/videos


Saving output3.mp4 to output3 (1).mp4
/content


In [ ]:
#@title ## Step 3: Noise Reduction

import cv2                               #for image operations
import numpy as np                       #for image stored in array
from matplotlib import pyplot as plt     #for plotting the graph
import math                              #for math iopoerations
import glob                              #to access the files from folder
import os                                #for file or folder handling
import shutil                            #to remove the folder with files

video_input_folder = 'output1' #@param 
video_name = 'input (online-video-cutter.com)_realsr_ncnn_vulkan_1920.mp4' #@param 
output_path = '/drive/MyDrive/' + video_input_folder + '/' + video_name
videopath = os.path.join(output_path)
print("Your Path is===",videopath)

#break video into frames with noisy frames and creating folder
!mkdir /content/frame_with_noise
path_noise = "/content/frame_with_noise"

vidcap1 = cv2.VideoCapture(videopath)    #capturing video
success,image1 = vidcap1.read()          #Read the video

#totalframecount= int(vidcap1.get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown Type initial video length
seconds = 2  #@param 
totalframecount = seconds*1000

# Capture frame-by-frame
count = 0
success = True

while success:
  #"imgWN" + "{:04d}".format(number) + '.jpg'
  cv2.imwrite("%s/imgN%04d.jpg" %(path_noise,count),image1)      # save frame as JPEG file
  vidcap1.set(cv2.CAP_PROP_POS_MSEC,(count)) #used to hold speed of frame generation
                                             #Current position of the video file in milliseconds
  
  success,image1 = vidcap1.read()
  print ('Read a new frame: ', success)
  count += 1
  if(count==totalframecount):
      break
print("the total frames=",count)
cv2.waitKey(0)  
 
# Conversion of the clear frames

#break video into frames with noise removal with filter
!mkdir /content/frame_without_noise
path_wo_noise = '/content/frame_without_noise'
if os.path.exists(path_wo_noise) and os.path.isdir(path_wo_noise):
    shutil.rmtree(path_wo_noise)
    print("Delete")
if not os.path.isfile(path_wo_noise):
    os.makedirs(path_wo_noise)
    print("Create")


vidcap = cv2.VideoCapture(videopath)
success,image = vidcap.read()#READ THE VIDEO
count1 = 0
success = True
while success:
  blur = cv2.bilateralFilter(image,9,75,75)  #filter
  opr=cv2.fastNlMeansDenoisingColored(blur,None,10,10,7,21)    #remove noise
  cv2.imwrite("%s/imgWN%04d.jpg" %(path_wo_noise,count1),opr)     # save frame as JPEG file
  vidcap.set(cv2.CAP_PROP_POS_MSEC,(count1)) #used to hold speed of frane generation
  success,image = vidcap.read()
  print ('Read a new frame: ', success)
  count1 += 1
  if(count1==totalframecount):
      break
  print("count 2====",count1)
cv2.waitKey(0)  



In [ ]:
#@title ## Save Denoised Video
import numpy as np
import glob


img_array = []
for filename in sorted(glob.glob('/content/frame_without_noise/*.jpg')):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

video_output_folder = "output1" #@param 
denoise_video_name = 'output3.mp4' #@param 
output_path2 = '/drive/MyDrive/' + video_output_folder + '/' + denoise_video_name
videopath2 = os.path.join(output_path)

#@markdown Type initial video length
seconds = 2  #@param 
fps = seconds*100

out = cv2.VideoWriter('/content/'+denoise_video_name,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


In [ ]:
#@title ## Step 4: Upscale Video

import os

#@markdown Choose output **folder** name:
output_folder = 'output1' #@param 

#@markdown Choose output **video resolution**:
W = 2160 #@param [720, 1080, 1440, 2160]
H = 3840 #@param [1280, 1920, 2560, 3840]
resol = [W, H]
alg = "realcugan_ncnn_vulkan" #@param ['realsr_ncnn_vulkan', 'waifu2x_ncnn_vulkan', 'realcugan_ncnn_vulkan']

input_path = '/content/videos'
output_path = '/drive/MyDrive/' + output_folder
!mkdir {output_path}

if os.path.isdir(output_path) is False:
  raise Exception("Output directory not created")

for filename in os.listdir(input_path):
  filepath = os.path.join(input_path, filename)
  if os.path.isfile(filepath):
    out_filename = pathlib.Path(filepath).stem + '_' + alg + '_' + str(resol[1]) + '.mp4'
    outPath = os.path.join(output_path, out_filename)
    !python video2x-5.0.0-beta5/video2x/video2x.py -i '{filepath}' -o '{outPath}' -d '{alg}' -h '{resol[0]}' #-c conf_files/video2x.yaml

print("*** Upscaled video is ready in the output folder ***")

mkdir: cannot create directory ‘/drive/MyDrive/output1’: File exists
python3: can't open file 'video2x-5.0.0-beta5/video2x/video2x.py': [Errno 2] No such file or directory
python3: can't open file 'video2x-5.0.0-beta5/video2x/video2x.py': [Errno 2] No such file or directory
*** Upscaled video is ready in the output folder ***
